In [1]:
import pandas as pd
import numpy as np

data17 = pd.read_csv("properati-AR-2017-08-01-properties-sell.csv", low_memory = False)
data16 = pd.read_csv("properati-AR-2016-12-01-properties-sell.csv", low_memory = False)
data15 = pd.read_csv("properati-AR-2015-12-01-properties-sell.csv", low_memory = False)
data14 = pd.read_csv("properati-AR-2015-12-01-properties-sell.csv", low_memory = False)
frames = [data14, data15, data16, data17]
data = pd.concat(frames)
data.head()

FileNotFoundError: File b'properati-AR-2017-08-01-properties-sell.csv' does not exist

In [ ]:
#Solo me quedo con CAP FED y GBA
dataCapGba = data.loc[(data.state_name.str.contains('Capital Federal') | data.state_name.str.contains('G.B.A') ), ['created_on', 'property_type', 'lat', 'lon', 'price', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms']]
dataCapGba = dataCapGba.loc[dataCapGba['price_usd_per_m2'] < 9000,:]
dataCapGba = dataCapGba.loc[dataCapGba['price_usd_per_m2'] > 1000,:]
dataCapGba = dataCapGba.loc[dataCapGba['surface_total_in_m2'] > 5, :]

In [ ]:
#Elimino duplicados y registros con ALGUNA COLUMNA NaN (no nos serviran para predecir)
dataCapGba.drop_duplicates(subset=['lat', 'lon', 'price_usd_per_m2'])
dataCapGba.dropna(inplace=True, how='any')
dataCapGba.head()

In [ ]:
# Importando librerias necesarias para ML
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
#Transformemos las columnas created_on y property_type a valores numericos para poder usarlos a la hora de predecir
#Para ello, crearemos algunas funciones diccionario que nos sirvan para dicha informacion:

#property_type's: apartament, house, PH, store
def tipoDePropiedad(x):
    tipos = {
        "apartment" : 1,
        "house" : 2,
        "PH" : 3,
        "store" : 4
    }
    return tipos[x] if x in tipos else x

from datetime import datetime
#created_on's: yyyy-mm-dd
def fechaNumerica(x):
    d = datetime.strptime('2012-02-10' , '%Y-%m-%d')
    return int("%4d%02d%02d" % (d.year, d.month, d.day)) + 0

In [ ]:
#Aplicamos las funciones anteriormente creadas
dataCapGba.loc[:, 'property_type'] = dataCapGba.property_type.apply(tipoDePropiedad)
dataCapGba.loc[:, 'created_on'] = dataCapGba.created_on.apply(fechaNumerica)

In [ ]:
#Ahora, todas las columnas son numericas
dataCapGba.info()

In [ ]:
#Convertimos todo a int
testData = dataCapGba
testData.loc[:, 'lat'] = testData.lat.apply(lambda x: int(x))
testData.loc[:, 'lon'] = testData.lon.apply(lambda x: int(x))
testData.loc[:, 'price'] = testData.price.apply(lambda x: int(x))
testData.loc[:, 'surface_total_in_m2'] = testData.surface_total_in_m2.apply(lambda x: int(x))
testData.loc[:, 'surface_covered_in_m2'] = testData.surface_covered_in_m2.apply(lambda x: int(x))
testData.loc[:, 'price_usd_per_m2'] = testData.price_usd_per_m2.apply(lambda x: int(x))
testData.loc[:, 'price_per_m2'] = testData.price_per_m2 .apply(lambda x: int(x))
testData.loc[:, 'rooms'] = testData.rooms.apply(lambda x: int(x))
SEED = 7

array = testData.values
X = array[:, :]
#Quiero estimar 'price_usd_per_m2'
Y = array[:, 7]
datosValidacion = 0.20
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=datosValidacion, random_state=SEED)

In [ ]:
testData.info()

In [ ]:
#Evaluando distintos algoritmos..
SCORING = 'accuracy'

aModelos = []
aModelos.append(('LR', LogisticRegression()))
aModelos.append(('LDA', LinearDiscriminantAnalysis()))
aModelos.append(('KNN', KNeighborsClassifier()))
aModelos.append(('CART', DecisionTreeClassifier()))
aModelos.append(('NB', GaussianNB()))
aModelos.append(('SVM', SVC()))
aResultados = []
aNombres = []
for nombre, modelo in aModelos:
    kfold = model_selection.KFold(n_splits=10, random_state=SEED)
    res = model_selection.cross_val_score(modelo, X_train, Y_train, cv=kfold, scoring=SCORING)
    aResultados.append(res)
    aNombres.append(nombre)
    print "%s: %f (%f)" % (nombre, res.mean(), res.std())